In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, preprocessing, metrics
df_train = pd.read_csv('../Data_with_NumMosquitos/unique_train.csv')
#df_train_nomosq = pd.read_csv('../train_filled_new.csv')
df_test = pd.read_csv('../Data_with_NumMosquitos/new_test_priors.csv')
#df_test_nomosq = pd.read_csv('../test_filled_new.csv')

In [2]:
random_state = 42
df_train = df_train.drop(['Block'],axis=1)
df_test = df_test.drop(['Block'],axis=1)

In [3]:
def year_train_test_split(train, target, year):

    # Create copy
    X = train.copy()

    # Retrieve target column and remove from X
    y = X[target]
    X.drop([target], axis=1)

    # Create mask
    msk = X.Year == year

    # Drop date column
    X = X.drop(['Year', 'WnvPresent_DateTrapSpecies'], axis=1)
    
    # Create dfs based on mask    
    X_train = X[~msk]
    X_test = X[msk]
    X_test = X_test.drop(['count_mosquitos'], axis=1)
    y_train = y[~msk]
    y_test = y[msk]
    y_train_numMosquitos = X.count_mosquitos[~msk] 
    y_test_numMosquitos = X.count_mosquitos[msk] 
    
    return X_train, X_test, y_train, y_test, y_train_numMosquitos, y_test_numMosquitos,msk

In [4]:
list1 = [#'NumMosquitos',
                            'Trap',
                            'Month',
                            'Block',
                            'AddressAccuracy',
                            'AddressNumberAndStreet',
                            'Address',
                            'Street',
                            'CodeSum'
]

[var for var in list1 if var in df_train.columns]

['Month', 'AddressAccuracy']

In [5]:
train_for_loo = df_train.drop(['Month', 'AddressAccuracy'], axis=1)
test_for_loo = df_test.drop(['Month','AddressAccuracy','Unnamed: 0','Year', 'count_mosquitos'],axis=1)
[var for var in test_for_loo.columns if var not in train_for_loo], 
[var for var in train_for_loo.columns if var not in test_for_loo]

['Year', 'count_mosquitos', 'WnvPresent_DateTrapSpecies']

In [6]:
y_clf = train_for_loo.WnvPresent_DateTrapSpecies
yhat_clf = pd.DataFrame(df_test.Tavg)
#yhat_clf.Tavg

#include Neural Net and XGBoost here!

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

#XgBoost
#NN

#clf = KNeighborsClassifier(n_neighbors=10)
clf_list = [RandomForestClassifier(n_estimators = 500,n_jobs = 3), GradientBoostingClassifier(),LogisticRegression()]
clf_names = ['RF','GBC','LogReg']

In [ ]:
#y_clf = train_for_loo.WnvPresent_DateTrapSpecies
#y_clf = 'NaN'

for clf,name in zip(clf_list,clf_names):
    for year in [2007, 2009, 2011, 2013]:

        X_train,X_test, y_train, y_test, y_train_numMosquitos, y_test_numMosquitos,year_mask = year_train_test_split(
            train_for_loo,
            'WnvPresent_DateTrapSpecies',
            year) 
        X_train = X_train.drop('count_mosquitos',axis=1)
    
        clf.fit(X_train,y_train)
        y_clf[year_mask] =clf.predict_proba(X_test)[:,1]
    #test3 = clf.predict_proba(X_train)
    #test2 = clf.predict_proba(X_test)
        yhat_clf[year]= clf.predict_proba(test_for_loo)[:,1]

#    test = clf.predict_proba(test_for_loo)
    try:
        yhat_clf = yhat_clf.drop('Tavg',axis=1)
    except: 
        print('already droped')
    df_train[name+'_prediction'] = y_clf
    df_test[name+'_prediction'] = yhat_clf.mean(axis=1)

In [ ]:
df_train.columns.names

In [19]:
clf = KNeighborsClassifier(n_neighbors=10)

In [20]:
for year in [2007, 2009, 2011, 2013]:

    X_train,X_test, y_train, y_test, y_train_numMosquitos, y_test_numMosquitos,year_mask = year_train_test_split(
            train_for_loo,
            'WnvPresent_DateTrapSpecies',
            year) 
    X_train = X_train.drop('count_mosquitos',axis=1)
    
    clf.fit(X_train,y_train)
    y_clf[year_mask] =clf.predict_proba(X_test)[:,1]
    #test3 = clf.predict_proba(X_train)
    #test2 = clf.predict_proba(X_test)
    yhat_clf[year]= clf.predict_proba(test_for_loo)[:,1]

#    test = clf.predict_proba(test_for_loo)
try:
    yhat_clf = yhat_clf.drop('Tavg',axis=1)
except: 
    print('already droped')

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
df_train['KNN_Prediction'] = y_clf
df_test['KNN_Prediction'] = yhat_clf.mean(axis=1)

In [ ]:
df_train.to_csv('train_plus_pred.csv',index=False)
df_test.to_csv('test_plus_pred.csv',index=False)